<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_03/class_003_image_processing_smart_trash_bin_006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 269MB/s]


In [4]:
FILE='trash_nov_22_2018.zip'

In [5]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [6]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [7]:
Img_Size = 200
ref = 'jpg'
notref = 'met'

In [8]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [9]:
def properties(glcm_names,img,Img_Size):
  glcm =[]
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# see https://github.com/ucfilho/ANN/blob/master/ANN_2022/Image/class_001_image_processing_smart_trash_bin_012.ipynb
features =[]
for i in range(len(img_name)):
  glcm = []
  img = X[i]
  glcm = properties(glcm_names,img,Img_Size)
  
  img = cv2.imread(img_name[i])
  green_img = img[:,:,1]
  values_green = properties(glcm_names,green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties(glcm_names,red_img,Img_Size)

  blue_img = img[:,:,0]
  values_blue = properties(glcm_names,blue_img,Img_Size)

  total= [*glcm,*values_red, *values_green, *values_blue]
  
  features.append(total)



In [10]:
lista = []
glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
names = glcm_names
for x in glcm_names:
  lista.append(x+'_r')
names = glcm_names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_g')
names = names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_b')
names = names + lista

print(names)

df = pd.DataFrame(features,columns=names)

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM', 'contrast_r', 'dissimilarity_r', 'homogeneity_r', 'energy_r', 'correlation_r', 'ASM_r', 'contrast_g', 'dissimilarity_g', 'homogeneity_g', 'energy_g', 'correlation_g', 'ASM_g', 'contrast_b', 'dissimilarity_b', 'homogeneity_b', 'energy_b', 'correlation_b', 'ASM_b']


In [11]:
glcm_names

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

In [12]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [13]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [14]:
df['target']=label

In [15]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [16]:
y=df['target']
X=df.drop(['target'], axis='columns')

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=1)

In [18]:
scaler = StandardScaler()
# transform data
scaler_X = scaler.fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

In [40]:
active='relu' # 'logistic' 
model = MLPClassifier(hidden_layer_sizes=(400,),solver='adam', 
                      activation= active,
                      random_state=1, max_iter=1000)
model.fit(X_train, y_train)
y_calc = model.predict(X_test)

In [38]:
confusion_matrix(y_test, y_calc)

array([[102,   8,   8,  12,   3,   0],
       [  3,  81,  30,  17,  30,   5],
       [  3,  31,  66,  15,   7,  13],
       [ 15,  16,   8, 142,  14,   1],
       [  3,  27,  15,  27,  85,   2],
       [  6,   4,   7,  11,   5,  12]])

In [39]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       0.77      0.77      0.77       133
       glass       0.49      0.49      0.49       166
       metal       0.49      0.49      0.49       135
       paper       0.63      0.72      0.68       196
     plastic       0.59      0.53      0.56       159
       trash       0.36      0.27      0.31        45

    accuracy                           0.59       834
   macro avg       0.56      0.54      0.55       834
weighted avg       0.58      0.59      0.58       834



In [22]:
''''
Img_Size = 80 hidden = 12

               precision    recall  f1-score   support

   cardboard       0.78      0.74      0.76       133
       glass       0.51      0.41      0.45       166
       metal       0.45      0.46      0.45       135
       paper       0.58      0.68      0.63       196
     plastic       0.49      0.59      0.54       159
       trash       0.47      0.18      0.26        45

    accuracy                           0.56       834
   macro avg       0.55      0.51      0.51       834
weighted avg       0.55      0.56      0.55       834

'''

"'\nImg_Size = 80\n\n               precision    recall  f1-score   support\n\n   cardboard       0.78      0.74      0.76       133\n       glass       0.51      0.41      0.45       166\n       metal       0.45      0.46      0.45       135\n       paper       0.58      0.68      0.63       196\n     plastic       0.49      0.59      0.54       159\n       trash       0.47      0.18      0.26        45\n\n    accuracy                           0.56       834\n   macro avg       0.55      0.51      0.51       834\nweighted avg       0.55      0.56      0.55       834\n\n"

In [23]:
'''
Img_Size = 100

               precision    recall  f1-score   support

   cardboard       0.79      0.74      0.76       133
       glass       0.45      0.40      0.42       166
       metal       0.45      0.44      0.44       135
       paper       0.59      0.66      0.62       196
     plastic       0.48      0.58      0.53       159
       trash       0.59      0.22      0.32        45

    accuracy                           0.55       834
   macro avg       0.56      0.51      0.52       834
weighted avg       0.55      0.55      0.54       834
'''

'\nImg_Size = 100\n\n               precision    recall  f1-score   support\n\n   cardboard       0.79      0.74      0.76       133\n       glass       0.45      0.40      0.42       166\n       metal       0.45      0.44      0.44       135\n       paper       0.59      0.66      0.62       196\n     plastic       0.48      0.58      0.53       159\n       trash       0.59      0.22      0.32        45\n\n    accuracy                           0.55       834\n   macro avg       0.56      0.51      0.52       834\nweighted avg       0.55      0.55      0.54       834\n'

In [ ]:
'''
Img_Size = 200
               precision    recall  f1-score   support

   cardboard       0.82      0.76      0.79       133
       glass       0.46      0.46      0.46       166
       metal       0.44      0.41      0.42       135
       paper       0.61      0.69      0.65       196
     plastic       0.51      0.55      0.53       159
       trash       0.36      0.18      0.24        45

    accuracy                           0.56       834
   macro avg       0.53      0.51      0.51       834
weighted avg       0.55      0.56      0.55       834
'''

In [24]:
# Classification Report
print('\n\n', classification_report(y_test, y_test, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       1.00      1.00      1.00       133
       glass       1.00      1.00      1.00       166
       metal       1.00      1.00      1.00       135
       paper       1.00      1.00      1.00       196
     plastic       1.00      1.00      1.00       159
       trash       1.00      1.00      1.00        45

    accuracy                           1.00       834
   macro avg       1.00      1.00      1.00       834
weighted avg       1.00      1.00      1.00       834

